# A Basic Trend-Following System for Futures (Multi Pass)

In this notebook we show a very simple strategy based on the crossing of two moving averages. The strategy can be used as starting point for developing more complex approaches. We use a multi-pass implementation which forbids forward looking and can be used for testing your idea.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable widget scrolling

We define a wrapper function for loading the data, and implement our strategy inside a function which returns allocation weights for a given point in time. Then we call the built-in backtesting function and perform the simulation avoiding potential looking-forward issues. Note that the performace of this system is not so good as the Sharpe ratio smaller than 1. If you can write a system with a positive Sharpe ratio larger than 1, save your result and click on the **Submit** button in your **Development** area!

In [ ]:
import xarray as xr

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata



def load_data(period):
    """Loads futures markets for the Classic Futures contest"""
    return qndata.futures.load_data(tail=period, dims=("time","field","asset"))



def strategy(data):
    """The system will go long when the slow-moving moving average is smaller than the
    fast-moving one; otherwise it will not be invested.
    """
    close= data.sel(field="close")
    ma_slow= close.rolling(time=200).mean().isel(time=-1)
    ma_fast= close.rolling(time=20).mean().isel(time=-1)
    return xr.where(ma_fast > ma_slow, 1, 0)



weights = qnbt.backtest(
    # Classic Futures contest:
    competition_type= "futures",
    load_data= load_data,
    # lookback in calendar days, it should be larger than the max. lookback used for
    # indicators, in this case 200 trading days:
    lookback_period=365,
    start_date= "2006-01-01",
    strategy= strategy
)